In [ ]:
import cv2
import numpy as np
# import timeit
from IPython.display import display, clear_output
import PIL.Image
import io
import utils

widthCard = 330
heightCard = 440


c:\Users\valen\Downloads\Pokemon-Card-Scanner-main\Pokemon-Card-Scanner-main


In [ ]:
cam = cv2.VideoCapture(1)
# background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, 
#                                        
# cv2.namedWindow("Canny Tuner", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Canny Tuner", 400, 100)
# cv2.createTrackbar("Threshold1", "Canny Tuner", 0, 150, lambda x: None)
# cv2.createTrackbar("Threshold2", "Canny Tuner", 150, 500, lambda x: None)


blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)

def biggestContour(contours, min_area=5000): ###10000 might be too large?
    largest = None
    max_area = 0
    approx_corners = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:  # Only consider contours with area larger than min_area
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

            if len(approx) == 4 and area > max_area:
                largest = contour
                max_area = area
                approx_corners = approx

    return approx_corners, max_area

try:
    while True:
        check, frame = cam.read()
        if not check:
            print("Camera feed not detected.")
            break

        # Rotate the frame to get the correct orientation
        rot90frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        rot90frame = cv2.rotate(rot90frame, cv2.ROTATE_180)        # Make the image grayscale
        grayFrame = cv2.cvtColor(rot90frame, cv2.COLOR_BGR2GRAY)
        blurredFrame = cv2.GaussianBlur(grayFrame, (3, 3), 0)

        # Apply background subtraction
        # fg_mask = background_subtractor.apply(blurredFrame)#!!!!!
        # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        # fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)


        # thresh1 = input()
        # thresh2 = input()
    
        # Edge detection
        edgedFrame = cv2.Canny(blurredFrame, 120, 255)
        kernel = np.ones((1,1))
        frameDial = cv2.dilate(edgedFrame, kernel, iterations=1)
        frameThreshold = cv2.erode(frameDial, kernel, iterations=1)

        # Find contours
        contours, _ = cv2.findContours(frameThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contourFrame = rot90frame.copy()
        cv2.drawContours(contourFrame, contours, -1, (0, 255, 0), 10)

        # Process biggest contour for card
        corners, maxArea = biggestContour(contours)
        matchingCard = np.zeros((heightCard, widthCard, 3), dtype=np.uint8)
        found, cardinfo = False, 0
        imgWarpColored = blackImg

        if len(corners) == 4:
            corners = [corners[i][0] for i in range(4)]
            bigContour = rot90frame.copy()
            cv2.drawContours(bigContour, [np.array(corners)], -1, (255, 0, 0), 10)
            pts1 = np.float32(corners)
            pts2 = np.float32([[0, 0], [widthCard, 0], [0, heightCard], [widthCard, heightCard]])
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(rot90frame, matrix, (widthCard, heightCard))
            
            # Assuming utils.findCard is a function that finds the card match, you may keep this line if needed
            # found, matchingCard, cardinfo = utils.findCard(imgWarpColored.copy())

        # Resize frames for display
        rot90frame = cv2.resize(rot90frame, (widthCard, heightCard))
        grayFrame = cv2.resize(grayFrame, (widthCard, heightCard))
        blurredFrame = cv2.resize(blurredFrame, (widthCard, heightCard))
        edgedFrame = cv2.resize(edgedFrame, (widthCard, heightCard))
        contourFrame = cv2.resize(contourFrame, (widthCard, heightCard))
        bigContour = cv2.resize(bigContour, (widthCard, heightCard)) if 'bigContour' in locals() else blackImg
        # fg_mask = cv2.resize(fg_mask, (widthCard, heightCard))

        # Stack and display images in Jupyter Notebook
        imageArr = ([rot90frame, grayFrame, blurredFrame, edgedFrame],
                    [contourFrame, bigContour, imgWarpColored, rot90frame])
        labels = [["Original", "Gray", "Blurred", "Threshold"],
                  ["Contours", "Biggest Contour", "Warped Perspective", "OG"]]
        
        # Assuming utils.makeDisplayImage creates a labeled display image; otherwise, you can replace it
        stackedImage = utils.makeDisplayImage(imageArr, labels)
        stackedImage = cv2.cvtColor(stackedImage, cv2.COLOR_BGR2RGB)
        
        # Convert and display
        pil_im = PIL.Image.fromarray(stackedImage)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        k = cv2.waitKey(1) & 0xFF
        if k==27:
            break
        clear_output(wait=True)
        display(PIL.Image.open(buf))

except KeyboardInterrupt:
    print("Stopped by user")
finally:
    cam.release()
    cv2.destroyAllWindows()


Stopped by user


In [ ]:
import cv2
import numpy as np
# import timeit
from IPython.display import display, clear_output
import PIL.Image
import io
import utils

widthCard = 330
heightCard = 440
cam = cv2.VideoCapture(1)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, 
                                                           varThreshold=20, detectShadows=False)

blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)

def biggestContour(contours, min_area=100000): ###10000 might be too large?
    largest = None
    max_area = 0
    approx_corners = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:  # Only consider contours with area larger than min_area
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

            if len(approx) == 4 and area > max_area:
                largest = contour
                max_area = area
                approx_corners = approx

    return approx_corners, max_area

try:
    while True:
        check, frame = cam.read()
        rot90frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        # Create a blank image
        blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)

        # Make image gray scale
        grayFrame =  cv2.cvtColor(rot90frame, cv2.COLOR_BGR2GRAY)
        # Blur the image to reduce noise
        blurredFrame = cv2.GaussianBlur(grayFrame, (3, 3), 0)

        # # Background Remover 
        # background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
        
        # fg_mask = background_subtractor.apply(blurredFrame)
        # foreground = cv2.bitwise_and(blurredFrame, blurredFrame, mask=fg_mask)

        # Use Canny edge detection to get edges
        edgedFrame = cv2.Canny(blurredFrame, threshold1=80, threshold2=150)
        # edgedFrame = cv2.adaptiveThreshold(blurredFrame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 303, 56)
        # Clean up edges
        kernel = np.ones((5,5))
        frameDial = cv2.dilate(edgedFrame, kernel, iterations=2)
        frameThreshold = cv2.erode(frameDial, kernel, iterations=1)

        # Get image contours
        contourFrame = rot90frame.copy()
        bigContour = rot90frame.copy()
        contours, hierarchy = cv2.findContours(edgedFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(contourFrame, contours, -1, (0, 255, 0), 10)

        imgWarpColored = blackImg  # Set imgWarpColored
        # Get biggest contour
        corners, maxArea = utils.biggestContour(contours)
        # Initialize values for matching card and found to ensure loop works even though no values are found. 
        matchingCard = np.zeros((heightCard, widthCard, 3), dtype=np.uint8)
        found = False #
        cardinfo = 0
        if len(corners) == 4:
            corners = [corners[0][0], corners[1][0], corners[2][0], corners[3][0]]
            corners = utils.reorderCorners(corners)  # Reorders corners to [topLeft, topRight, bottomLeft, bottomRight]
            #cv2.drawContours(bigContour, corners, -1, (0, 255, 0), 10)
            bigContour = utils.drawRectangle(bigContour, corners)
            pts1 = np.float32(corners)
            pts2 = np.float32([[0, 0], [widthCard, 0], [0, heightCard], [widthCard, heightCard]])
            # Makes a matrix that transforms the detected card to a vertical rectangle
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            # Transforms card to a rectangle widthCard x heightCard
            imgWarpColored = cv2.warpPerspective(rot90frame, matrix, (widthCard, heightCard))
            section_start = timeit.default_timer()
            # Check if a matching card has been found, and if so, display it
            found, matchingCard, cardinfo = utils.findCard(imgWarpColored.copy())  # Check to see if a matching card was found
            section_end = timeit.default_timer()            

        # Resize all of the images to the same dimensions
        # Note: imgWarpColored is already resized and matchingCard gets resized in utils.getMatchingCard()
        rot90frame = cv2.resize(rot90frame, (widthCard, heightCard))
        grayFrame = cv2.resize(grayFrame, (widthCard, heightCard))
        blurredFrame = cv2.resize(blurredFrame, (widthCard, heightCard))
        edgedFrame = cv2.resize(edgedFrame, (widthCard, heightCard))
        contourFrame = cv2.resize(contourFrame, (widthCard, heightCard))
        bigContour = cv2.resize(bigContour, (widthCard, heightCard))
        

        # print(f"Time to read and rotate image: {section_end - section_start:.4f} seconds")

        # An array of all 8 images
        imageArr = ([rot90frame, grayFrame, blurredFrame, edgedFrame],
                    [contourFrame, bigContour, imgWarpColored, matchingCard])

        # Labels for each image
        labels = [["Original", "Gray", "Blurred", "Threshold"],
                  ["Contours", "Biggest Contour", "Warped Perspective", "Matching Card"]]

        # Stack all 8 images into one and add text labels
        stackedImage = utils.makeDisplayImage(imageArr, labels)
        # Convert and display
        pil_im = PIL.Image.fromarray(stackedImage)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        
        clear_output(wait=True)
        display(PIL.Image.open(buf))
except KeyboardInterrupt:
    print("Stopped by user")
finally:
    cam.release()
    cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import sys
print(sys.executable)

C:\Users\valen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [ ]:
import cv2
import numpy as np
# import timeit
from IPython.display import display, clear_output
import PIL.Image
import io
import utils

widthCard = 330
heightCard = 440
cam = cv2.VideoCapture(1)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, 
                                                           varThreshold=20, detectShadows=False)

blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)

def biggestContour(contours, min_area=100000): ###10000 might be too large?
    largest = None
    max_area = 0
    approx_corners = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:  # Only consider contours with area larger than min_area
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

            if len(approx) == 4 and area > max_area:
                largest = contour
                max_area = area
                approx_corners = approx

    return approx_corners, max_area

try:
    while True:
        check, frame = cam.read()
        if not check:
            break
        
        rot90frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        grayFrame = cv2.cvtColor(rot90frame, cv2.COLOR_BGR2GRAY)
        
        # Adaptive Histogram Equalization for contrast enhancement
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhancedGray = clahe.apply(grayFrame)
        
        # Gaussian Blur to reduce noise
        blurredFrame = cv2.GaussianBlur(enhancedGray, (7, 7), 0)

        # Background Subtraction
        fg_mask = background_subtractor.apply(blurredFrame)
        foreground = cv2.bitwise_and(blurredFrame, blurredFrame, mask=fg_mask)

        # Edge Detection (Canny)
        edgedFrame = cv2.Canny(foreground, threshold1=50, threshold2=150)
        
        # Morphological Operations for cleaner edges
        kernel = np.ones((5, 5))
        frameDial = cv2.dilate(edgedFrame, kernel, iterations=2)
        frameThreshold = cv2.erode(frameDial, kernel, iterations=1)

        # Contour Detection
        contours, _ = cv2.findContours(frameThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contourFrame = rot90frame.copy()

        # Find the biggest contour
        corners, maxArea = biggestContour(contours, min_area=5000)  # Adjust min_area as needed

        if len(corners) == 4:
            corners = [corners[0][0], corners[1][0], corners[2][0], corners[3][0]]
            corners = utils.reorderCorners(corners)
            bigContour = utils.drawRectangle(contourFrame, corners)
            pts1 = np.float32(corners)
            pts2 = np.float32([[0, 0], [widthCard, 0], [0, heightCard], [widthCard, heightCard]])

            # Perspective Transform
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(rot90frame, matrix, (widthCard, heightCard))

            # Match the card
            found, matchingCard, cardinfo = utils.findCard(imgWarpColored.copy())

        else:
            bigContour = rot90frame.copy()  # If no contour is found, show original frame

        # Display results (resize images as needed)
        cv2.imshow("Contours", bigContour)
        cv2.imshow("Warped", imgWarpColored)

        # An array of all 8 images
        imageArr = ([contourFrame, bigContour, imgWarpColored, matchingCard])

        # Labels for each image
        labels = [["Contours", "Biggest Contour", "Warped Perspective", "Matching Card"]]

        # Stack all 8 images into one and add text labels
        stackedImage = utils.makeDisplayImage(imageArr, labels)
        # Convert and display
        pil_im = PIL.Image.fromarray(stackedImage)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        
        clear_output(wait=True)
        display(PIL.Image.open(buf))


except KeyboardInterrupt:
    print("Stopped by user")
finally:
    cam.release()
    cv2.destroyAllWindows()


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1920 and the array at index 2 has size 1320

ChatGPT Attempt to fix bg stuff
